In [1]:
import cv2
import os
import shutil
import numpy as np
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from PIL import Image, ImageFilter, ImageEnhance
from keras.optimizers import Adam
from keras.regularizers import l2

In [2]:
# Unzipping the image dataset
zip_ref = zipfile.ZipFile('original.zip', 'r')
zip_ref.extractall('dataset')  # Extracting to the 'dataset' folder
zip_ref.close()

In [3]:
# BW
datagen = ImageDataGenerator()

bw_dir = 'dataset/BW'
if not os.path.exists(bw_dir):
    os.makedirs(bw_dir)

original_dir = 'dataset/original'
total_images = 20

for i in range(1, total_images + 1):
    img_path = f'{original_dir}/{i:02d}.jpg'
    img = load_img(img_path, color_mode='grayscale')
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    j = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=bw_dir, save_prefix='bw', save_format='jpeg'):
        j += 1
        if j >= 10:
            break

In [4]:
# Define data paths
original_dir = 'dataset/original'
BW_dir = 'dataset/BW'
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Create directories for training and validation
for category in ['original', 'BW']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)

# Function to split data and move files
def split_data(source_folder, train_folder, validation_folder, validation_split=0.2):
    files = os.listdir(source_folder)
    train_files, validation_files = train_test_split(files, test_size=validation_split)

    # Copy files to training directory
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(train_folder, file))

    # Copy files to validation directory
    for file in validation_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(validation_folder, file))

# Apply function to original and distorted datasets
split_data(original_dir, os.path.join(train_dir, 'original'), os.path.join(validation_dir, 'original'))
split_data(BW_dir, os.path.join(train_dir, 'BW'), os.path.join(validation_dir, 'BW'))

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

Found 174 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [6]:
dataset_dir = 'dataset'
print("Directories in dataset:", os.listdir(dataset_dir))

for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if os.path.isdir(category_path):
        print(f"Contents of {category}:", os.listdir(category_path))

Directories in dataset: ['BW', 'original', 'validation', 'train']
Contents of BW: ['bw_0_5846.jpeg', 'bw_0_2795.jpeg', 'bw_0_7569.jpeg', 'bw_0_7401.jpeg', 'bw_0_9843.jpeg', 'bw_0_7188.jpeg', 'bw_0_3232.jpeg', 'bw_0_1602.jpeg', 'bw_0_6902.jpeg', 'bw_0_5316.jpeg', 'bw_0_9990.jpeg', 'bw_0_1616.jpeg', 'bw_0_5309.jpeg', 'bw_0_238.jpeg', 'bw_0_3036.jpeg', 'bw_0_472.jpeg', 'bw_0_8709.jpeg', 'bw_0_6186.jpeg', 'bw_0_5067.jpeg', 'bw_0_9448.jpeg', 'bw_0_8915.jpeg', 'bw_0_331.jpeg', 'bw_0_9090.jpeg', 'bw_0_8871.jpeg', 'bw_0_8245.jpeg', 'bw_0_6728.jpeg', 'bw_0_8495.jpeg', 'bw_0_2005.jpeg', 'bw_0_1794.jpeg', 'bw_0_8977.jpeg', 'bw_0_7063.jpeg', 'bw_0_4135.jpeg', 'bw_0_3663.jpeg', 'bw_0_3492.jpeg', 'bw_0_8094.jpeg', 'bw_0_4566.jpeg', 'bw_0_8677.jpeg', 'bw_0_9537.jpeg', 'bw_0_2590.jpeg', 'bw_0_3206.jpeg', 'bw_0_3545.jpeg', 'bw_0_1469.jpeg', 'bw_0_890.jpeg', 'bw_0_7475.jpeg', 'bw_0_6781.jpeg', 'bw_0_7257.jpeg', 'bw_0_7592.jpeg', 'bw_0_3254.jpeg', 'bw_0_5353.jpeg', 'bw_0_2884.jpeg', 'bw_0_4678.jpeg', 'bw

In [7]:
import shutil
import os

directories_to_remove = ['original', 'BW']
base_dir = 'dataset'

for dir_name in directories_to_remove:
    dir_path = os.path.join(base_dir, dir_name)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed directory: {dir_path}")
    else:
        print(f"Directory does not exist, no need to remove: {dir_path}")

print("Updated contents of the dataset directory:", os.listdir(base_dir))

Removed directory: dataset/original
Removed directory: dataset/BW
Updated contents of the dataset directory: ['validation', 'train']


In [8]:
import shutil

def remove_unwanted_dirs(directory):
    checkpoint_dir = os.path.join(directory, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
        print(f"Removed {checkpoint_dir}")

remove_unwanted_dirs(train_dir)
remove_unwanted_dirs(validation_dir)

print("Updated training directories:", os.listdir(train_dir))
print("Updated validation directories:", os.listdir(validation_dir))

Updated training directories: ['BW', 'original']
Updated validation directories: ['BW', 'original']


In [9]:
!pip install tensorflow_addons

import tensorflow_addons as tfa

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # L2
x = Dropout(0.5)(x)  # Dropout
predictions = Dense(1, activation='sigmoid')(x)

optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.001)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Prepare training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # Use 20% of the data as validation set
)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='validation'
)

# Use early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 10.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


58889256/58889256 [==============================] - 0s 0us/step
Found 176 images belonging to 2 classes.
Found 42 images belonging to 2 classes.


In [10]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=10,
    callbacks=[early_stopping],
    validation_data=validation_generator,
    validation_steps=5
)

# Evaluate the model
results = model.evaluate(validation_generator)
print("Test loss, Test accuracy:", results)

Epoch 1/10
15/15 [==============================] - 84s 5s/step - loss: 7.9526 - accuracy: 0.6400 - val_loss: 8.5033 - val_accuracy: 0.8000
Epoch 2/10
15/15 [==============================] - 70s 5s/step - loss: 10.3472 - accuracy: 0.6667 - val_loss: 11.9309 - val_accuracy: 0.8000
Epoch 3/10
15/15 [==============================] - 70s 5s/step - loss: 11.9537 - accuracy: 0.7333 - val_loss: 13.7020 - val_accuracy: 0.7200
Epoch 4/10
15/15 [==============================] - 66s 4s/step - loss: 9.1250 - accuracy: 0.5867 - val_loss: 13.8132 - val_accuracy: 0.6800
Epoch 5/10
15/15 [==============================] - 69s 5s/step - loss: 4.5765 - accuracy: 0.8169 - val_loss: 5.3265 - val_accuracy: 0.6800
Epoch 6/10
15/15 [==============================] - 67s 5s/step - loss: 6.6657 - accuracy: 0.6761 - val_loss: 8.3294 - val_accuracy: 0.6400
Epoch 7/10
15/15 [==============================] - 70s 5s/step - loss: 4.4667 - accuracy: 0.6479 - val_loss: 8.2524 - val_accuracy: 0.7200
Epoch 8/10
15/1